In [5]:
%pip uninstall pyvis -y
%pip install pyvis==0.1.9

Found existing installation: pyvis 0.1.9
Uninstalling pyvis-0.1.9:
  Successfully uninstalled pyvis-0.1.9
Note: you may need to restart the kernel to use updated packages.
  Using cached pyvis-0.1.9-py3-none-any.whl.metadata (384 bytes)
Using cached pyvis-0.1.9-py3-none-any.whl (23 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pyvis.network import Network
import networkx as nx
import pandas as pd

# Load your saved graph
G = nx.read_gexf("nz_legislation_with_adjusted_communities.gexf")

# Initialize Pyvis
net = Network(height="800px", width="100%", notebook=False)

# Add nodes with community-based grouping
for node, data in G.nodes(data=True):
    label = data.get("name", node)
    community = data.get("community", "unknown")
    net.add_node(node, label=label, title=f"Community: {community}", group=community)

# Add edges
for u, v in G.edges():
    net.add_edge(u, v)

#net.barnes_hut()  # stabilizes the layout and prevents excessive movement
# Before show() add this:
net.set_options("""
{
  "physics": {
    "stabilization": {
      "enabled": true,
      "iterations": 1000,
      "updateInterval": 25
    },
    "barnesHut": {
      "gravitationalConstant": -8000,
      "centralGravity": 0.3,
      "springLength": 95,
      "springConstant": 0.04,
      "damping": 0.09,
      "avoidOverlap": 1
    }
  }
}
""")

# Save and open interactive HTML
net.show("legislation_communities3.html")

In [5]:
# Count unique community IDs
communities = set(nx.get_node_attributes(G, "community").values())
print(f"Number of communities: {len(communities)}")

Number of communities: 14


In [6]:
from collections import Counter

community_sizes = Counter(nx.get_node_attributes(G, "community").values())

# Sort by size
for comm_id, size in sorted(community_sizes.items(), key=lambda x: -x[1]):
    print(f"Community {comm_id}: {size} acts")

Community 10: 292 acts
Community 15: 263 acts
Community 20: 218 acts
Community 7: 197 acts
Community 8: 156 acts
Community 9: 146 acts
Community 12: 124 acts
Community 16: 113 acts
Community 27: 106 acts
Community 11: 99 acts
Community 4: 89 acts
Community 3: 55 acts
Community 22: 55 acts
Community 6: 51 acts


In [12]:
import networkx as nx
from collections import defaultdict, Counter
from pyvis.network import Network

# Load your full act-to-act graph (with 'community' as node attribute)
G = nx.read_gexf("nz_legislation_with_adjusted_communities.gexf")

# Step 1: Count number of acts in each community
community_sizes = Counter(nx.get_node_attributes(G, "community").values())

# Step 2: Build a new projection graph
C = nx.Graph()

# Step 3: Count cross-community edges
edge_weights = defaultdict(int)

for u, v in G.edges():
    cu = G.nodes[u].get("community")
    cv = G.nodes[v].get("community")
    if cu != cv:
        key = tuple(sorted((cu, cv)))
        edge_weights[key] += 1

# Step 4: Add community nodes with size
for comm, size in community_sizes.items():
    C.add_node(comm, size=size)

# Step 5: Add weighted edges between communities
for (comm1, comm2), weight in edge_weights.items():
    C.add_edge(comm1, comm2, weight=weight)

# Step 6: Visualize it with Pyvis
net = Network(height="700px", width="100%", notebook=False)
for node, data in C.nodes(data=True):
    net.add_node(str(node), label=f"Community {node}", size=data["size"] * 2)

for u, v, data in C.edges(data=True):
    net.add_edge(str(u), str(v), value=data["weight"])

net.show("community_projection_only_communities.html")
print("✅ Projection graph saved as community_projection.html")

✅ Projection graph saved as community_projection.html


In [13]:
import networkx as nx
from collections import defaultdict, Counter
from pyvis.network import Network

# Load your act-level graph (with community labels)
G = nx.read_gexf("nz_legislation_with_adjusted_communities.gexf")

# Step 1: Count how many acts in each community
community_sizes = Counter(nx.get_node_attributes(G, "community").values())

# Step 2: Build a new projection graph
C = nx.Graph()

# Step 3: Count edges between communities
edge_weights = defaultdict(int)
for u, v in G.edges():
    cu = G.nodes[u].get("community")
    cv = G.nodes[v].get("community")
    if cu != cv:
        key = tuple(sorted((cu, cv)))
        edge_weights[key] += 1

# Step 4: Add nodes (one per community) with act counts
for comm, size in community_sizes.items():
    C.add_node(comm, size=size)

# Step 5: Add weighted edges between communities
for (comm1, comm2), weight in edge_weights.items():
    C.add_edge(comm1, comm2, weight=weight)

# Step 6: Visualize with Pyvis
net = Network(height="700px", width="100%", notebook=False)
net.barnes_hut()

# Color map for distinct community colors (optional: use your own palette)
from matplotlib import cm
import matplotlib.colors as mcolors

color_map = cm.get_cmap('tab20', len(C.nodes))  # up to 20 distinct colors
community_colors = {
    community: mcolors.to_hex(color_map(i)) for i, community in enumerate(sorted(C.nodes()))
}

# Add community nodes with color and scaled size
for node, data in C.nodes(data=True):
    size = data["size"]
    color = community_colors.get(node, "#cccccc")
    net.add_node(
        str(node),
        label=f"Community {node}",
        title=f"{size} Acts",
        size=size * 0.2,
        color=color
    )

# Add edges between communities with scaled thickness
for u, v, data in C.edges(data=True):
    weight = data["weight"]
    net.add_edge(
        str(u), str(v),
        value=max(1, weight // 5),
        title=f"{weight} links"
    )

# Export to HTML
net.show("community_projection_colored.html")
print("✅ Saved: community_projection_colored.html")

✅ Saved: community_projection_colored.html


/var/folders/9y/3rlj5x9s12b4rh6g81wmnvfc0000gp/T/ipykernel_39368/183491769.py:39: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  color_map = cm.get_cmap('tab20', len(C.nodes))  # up to 20 distinct colors


In [2]:
import networkx as nx
from collections import defaultdict, Counter
from pyvis.network import Network
import pandas as pd
from matplotlib import cm
import matplotlib.colors as mcolors

# === Step 1: Load full act-level graph ===
G = nx.read_gexf("nz_legislation_with_adjusted_communities.gexf")

# === Step 2: Load topic label mapping ===
# Your CSV must have columns: 'Topic' and 'topic_label'
df_labels = pd.read_csv("topic_similarity_scores_external.csv")
topic_map = dict(zip(df_labels["Topic"], df_labels["topic_label"]))

# === Step 3: Count acts per community ===
community_sizes = Counter(nx.get_node_attributes(G, "community").values())

# === Step 4: Build the projection graph (one node per community) ===
C = nx.Graph()

# Count number of cross-community relationships
edge_weights = defaultdict(int)
for u, v in G.edges():
    cu = G.nodes[u].get("community")
    cv = G.nodes[v].get("community")
    if cu != cv:
        key = tuple(sorted((cu, cv)))
        edge_weights[key] += 1

# Add community nodes with topic labels and size
for comm_id, size in community_sizes.items():
    label = topic_map.get(int(comm_id), f"Community {comm_id}")
    C.add_node(comm_id, size=size, label=label)

# Add cross-community edges
for (comm1, comm2), weight in edge_weights.items():
    C.add_edge(comm1, comm2, weight=weight)

# === Step 5: Visualize using Pyvis ===
net = Network(height="700px", width="100%", notebook=False)
net.barnes_hut()

# Generate a distinct color for each community
color_map = cm.get_cmap('tab20', len(C.nodes))
community_colors = {
    community: mcolors.to_hex(color_map(i)) for i, community in enumerate(sorted(C.nodes()))
}

# Add nodes with colored labels and tooltips
for node, data in C.nodes(data=True):
    label = data["label"]
    size = data["size"]
    color = community_colors.get(node, "#cccccc")

    net.add_node(
        str(node),
        label=label,
        title=f"{label}<br>{size} Acts",
        size=size * 0.2,
        color=color
    )

# Add visible weighted edges
for u, v, data in C.edges(data=True):
    weight = data["weight"]
    net.add_edge(
        str(u), str(v),
        value=max(1, weight // 5),
        title=f"{weight} cross-links"
    )

# Save and open interactive visual
net.show("community_projection_labeled.html")
print("✅ Saved: community_projection_labeled.html")

✅ Saved: community_projection_labeled.html


/var/folders/9y/3rlj5x9s12b4rh6g81wmnvfc0000gp/T/ipykernel_41137/4091185314.py:45: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  color_map = cm.get_cmap('tab20', len(C.nodes))
